In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import torch.nn as nn
import torch.optim as optim
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)
transform = transforms.Compose(
    [transforms.RandomCrop(32, padding=4),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True) 

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=16,shuffle=False)

cpu


100%|██████████| 170498071/170498071 [00:13<00:00, 12458414.28it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
model = torchvision.models.alexnet(pretrained=True)

/Users/juniverse/opt/anaconda3/envs/veda/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/juniverse/opt/anaconda3/envs/veda/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /Users/juniverse/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:03<00:00, 67.4MB/s] 


In [4]:
num_ftrs = model.classifier[6].in_features # fc의 입력 노드 수를 산출 
model.features[0] = nn.Conv2d(3, 64, kernel_size=5, stride=1)
model.classifier[6] = nn.Linear(num_ftrs, 10) # fc를 nn.Linear(num_ftrs, 10)로 대체
model = model.to(device) # 출력층 확인하기

In [10]:
i = 0
for name, param in model.named_parameters():  # named_parameters 가중치 뽑아주는 함수
    
    print(i,name," : ")
    print(param.shape)
    i+= 1

0 features.0.weight  : 
torch.Size([64, 3, 5, 5])
1 features.0.bias  : 
torch.Size([64])
2 features.3.weight  : 
torch.Size([192, 64, 5, 5])
3 features.3.bias  : 
torch.Size([192])
4 features.6.weight  : 
torch.Size([384, 192, 3, 3])
5 features.6.bias  : 
torch.Size([384])
6 features.8.weight  : 
torch.Size([256, 384, 3, 3])
7 features.8.bias  : 
torch.Size([256])
8 features.10.weight  : 
torch.Size([256, 256, 3, 3])
9 features.10.bias  : 
torch.Size([256])
10 classifier.1.weight  : 
torch.Size([4096, 9216])
11 classifier.1.bias  : 
torch.Size([4096])
12 classifier.4.weight  : 
torch.Size([4096, 4096])
13 classifier.4.bias  : 
torch.Size([4096])
14 classifier.6.weight  : 
torch.Size([10, 4096])
15 classifier.6.bias  : 
torch.Size([10])


In [11]:

for i, (name, param) in enumerate(model.named_parameters()):
    
    param.requires_grad = False
    if i == 9:
        print('end')
        break  

end


In [13]:
for i, (name, param) in enumerate(model.named_parameters()):
    print(param.requires_grad)

False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True


In [17]:
from tqdm import tqdm
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-2)
for epoch in range(1):

    running_loss = 0.0
    # for data in tqdm(trainloader):
    for data in tqdm(testloader):
        
        inputs, labels = data[0].to(device), data[1].to(device)
          
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # cost = running_loss / len(trainloader)        
    cost = running_loss / len(testloader)        
    print('[%d] loss: %.3f' %(epoch + 1, cost))  
   

print('Finished Training')

100%|██████████| 625/625 [04:41<00:00,  2.22it/s]

[1] loss: 1.838
Finished Training


In [18]:
correct = 0
total = 0
with torch.no_grad():
    model.eval()
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

#result
# Accuracy of the network on the 10000 test images: 39 %

Accuracy of the network on the 10000 test images: 37 %


In [32]:
for para in model.parameters():
    print(para.shape)
    break

for name,param in model.named_parameters():
    print(name)
    print(param.shape)
    break
    

torch.Size([64, 3, 5, 5])
features.0.weight
torch.Size([64, 3, 5, 5])
